# Análisis de dataset Retail Sales Dataset (2023-2024)

Este DataSet se puede encontrar en Kaggle: https://www.kaggle.com/datasets/mohammadtalib786/retail-sales-dataset

### Lectura inicial de los datos

In [3]:
import numpy as np


def cargar_datos(ruta_archivo):
    datos = np.genfromtxt(
        ruta_archivo,
        delimiter=",",
        skip_header=1,
        # con dtype es posible indicar a numpy de qué tipo son cada una
        # de las columnas.
        dtype=[
            ('Transaction ID', 'i4'),
            ('Date', 'U10'),
            ('Customer ID', 'U10'),
            ('Gender', 'U10'),
            ('Age', 'i4'),
            ('Product Category', 'U10'),
            ('Quantity', 'i4'),
            ('Price per Unit', 'i4'),
            ('Total Amount', 'i4')
        ]
    )
    return datos


def verificar_datos_correctos(datos):
    # TODO: Corroborar con profesores si no se puede hacer este análisis directamente con numpy
    for fila in datos:
        for elemento in fila:
            if elemento is None or elemento == "":
                return False

    return True

datos = cargar_datos("../data/retail_sales_dataset.csv")
datos_correctos = verificar_datos_correctos(datos)

if not datos_correctos:
    print("Datos Incorrectos, verifique el dataset.")
else:
    print("Datos Correctos")

Datos Correctos


### Calcular el total de ventas por categoría de producto.

In [4]:
def obtener_total_ventas_categoria_producto(datos):
    categorias = np.unique(datos["Product_Category"])

    ventas_por_categoria = {}

    for categoria in categorias:
        ventas_por_categoria[categoria] = np.sum(datos[datos["Product_Category"] == categoria]["Total_Amount"])

    return ventas_por_categoria


ventas_por_categoria = obtener_total_ventas_categoria_producto(datos)

print("Ventas por categoría: \n")
for categoria, total in ventas_por_categoria.items():
    print(f"- {categoria}: {total}")


Ventas por categoría: 

- Beauty: 143515
- Clothing: 155580
- Electronic: 156905


### Calcular el promedio de ventas diarias por categoría de producto.

In [28]:
def sumar_totales_por_dia(datos):
    # Para almacenar el dataset de los datos sumados por día,
    # se usa un diccionario en donde la clave será una combinación
    # de la fecha y la categoría, para asegurarse de que solo
    # haya un valor por día y por categoría en el dataset resultante.
    datos_sumados_por_categoria = {}

    for fila in datos:
        # Sacamoms la fecha y la categoría de la tupla.
        fecha = fila[1]
        categoria = fila[5]

        # Creamos la clave por estos datos.
        clave = f"{fecha}_{categoria}"

        # En este código, si es que no existe la clave en el
        # diccionario, se inicializa con los campos Date, Product_Category y Total_Amount.
        # Caso contrario, este bloque de código ya no se ejecutará.
        if clave not in datos_sumados_por_categoria:
            datos_sumados_por_categoria[clave] = {
                "Date": str(fecha),
                "Product_Category": str(categoria),
                # El valor Total_Amount en este caso se utilizará como sumador,
                # por lo que se inicializa en 0.
                "Total_Amount": 0,
            }
        
        # Después del código previo, es seguro que existirá una entrada
        # de datos para clave. Se procede a sumar el valor de Total_Amount en el 
        # dataset a la suma
        datos_sumados_por_categoria[clave]["Total_Amount"] += float(fila[8])

    # Se aplanan los datos para luego convertirlos a un array numpy.
    datos_aplanados = [
        (valor["Date"], valor["Product_Category"], valor["Total_Amount"])
        for _, valor in datos_sumados_por_categoria.items()
    ]
    
    # Se crea un array numpy indicando los tipos de datos en el array.
    return np.array(
        datos_aplanados,
        dtype=[
            ('Date', 'U10'),
            ('Product_Category', 'U10'),
            ('Total_Amount', 'i4')
        ]
    )


def obtener_promedio_diario_ventas_por_categoria_producto(datos):
    # Es necesario sumar los datos por día y por categoría.
    datos_sumados_por_dia = sumar_totales_por_dia(datos)

    # Se sacan las categorías que son únicas en los datos.
    categorias = np.unique(datos_sumados_por_dia["Product_Category"])

    promedios_por_categoria = {}

    for categoria in categorias:
        # Se filtra los datos diarios por categoría y se saca el promedio.
        datos_categoria = datos_sumados_por_dia[datos_sumados_por_dia["Product_Category"] == categoria]
        promedio = np.mean(datos_categoria["Total_Amount"])

        promedios_por_categoria[categoria] = promedio

    return promedios_por_categoria
    

sumas = obtener_promedio_diario_ventas_por_categoria_producto(datos)
promedios_por_categoria = obtener_promedio_diario_ventas_por_categoria_producto(datos)

print("Promedios diarios por categoría: \n")
for categoria, promedio in promedios_por_categoria.items():
    print(f"- {categoria}: {promedio}")


Promedios diarios por categoría: 

- Beauty: 703.5049019607843
- Clothing: 670.6034482758621
- Electronic: 716.4611872146119


### Identifica las categorías de productos con mayores y menores ventas

In [21]:
def obtener_categoria_con_mayores_ventas(ventas_por_categoria):

    nombre_mayor_categoria = None
    cant_mayor_categoria = None

    for categoria, total in ventas_por_categoria.items():
        if nombre_mayor_categoria is None:
            nombre_mayor_categoria = categoria
            cant_mayor_categoria = total

        else:
            if total > cant_mayor_categoria:
                nombre_mayor_categoria = categoria
                cant_mayor_categoria = total

    return nombre_mayor_categoria, cant_mayor_categoria


def obtener_categoria_con_menores_ventas(ventas_por_categoria):

    nombre_menor_categoria = None
    cant_menor_categoria = None

    for categoria, total in ventas_por_categoria.items():
        if nombre_menor_categoria is None:
            nombre_menor_categoria = categoria
            cant_menor_categoria = total

        else:
            if total < cant_menor_categoria:
                nombre_menor_categoria = categoria
                cant_menor_categoria = cant_menor_categoria

    return nombre_menor_categoria, cant_menor_categoria


nombre_mayor_categoria, cant_mayor_categoria = obtener_categoria_con_mayores_ventas(ventas_por_categoria)
nombre_menor_categoria, cant_menor_categoria = obtener_categoria_con_menores_ventas(ventas_por_categoria)

print(f"Categoría con mayores ventas: {nombre_mayor_categoria}, cant: {cant_mayor_categoria}")
print(f"Categoría con menores ventas: {nombre_menor_categoria}, cant: {cant_menor_categoria}")

Categoría con mayores ventas: Electronic, cant: 156905
Categoría con menores ventas: Beauty, cant: 143515


### Manipulación de Datos

In [116]:
print("Filtro de datos sólo Electronic")
print(datos[datos["Product_Category"] == 'Electronic'])

Filtro de datos sólo por categorías
[(   3, '2023-01-13', 'CUST003', 'Male', 50, 'Electronic', 1,  30,   30)
 (   8, '2023-02-22', 'CUST008', 'Male', 30, 'Electronic', 4,  25,  100)
 (   9, '2023-12-13', 'CUST009', 'Male', 63, 'Electronic', 2, 300,  600)
 (  13, '2023-08-05', 'CUST013', 'Male', 22, 'Electronic', 3, 500, 1500)
 (  15, '2023-01-16', 'CUST015', 'Female', 42, 'Electronic', 4, 500, 2000)
 (  18, '2023-04-30', 'CUST018', 'Female', 47, 'Electronic', 2,  25,   50)
 (  26, '2023-10-07', 'CUST026', 'Female', 28, 'Electronic', 2, 500, 1000)
 (  29, '2023-08-18', 'CUST029', 'Female', 42, 'Electronic', 1,  30,   30)
 (  31, '2023-05-23', 'CUST031', 'Male', 44, 'Electronic', 4, 300, 1200)
 (  33, '2023-03-23', 'CUST033', 'Female', 50, 'Electronic', 2,  50,  100)
 (  45, '2023-07-03', 'CUST045', 'Female', 55, 'Electronic', 1,  30,   30)
 (  46, '2023-06-26', 'CUST046', 'Female', 20, 'Electronic', 4, 300, 1200)
 (  48, '2023-05-16', 'CUST048', 'Male', 54, 'Electronic', 3, 300,  900)
 

In [117]:
print("Filtro de datos sólo Beauty")
print(datos[datos["Product_Category"] == 'Beauty'])

Filtro de datos sólo Beauty
[(  1, '2023-11-24', 'CUST001', 'Male', 34, 'Beauty', 3,  50,  150)
 (  5, '2023-05-06', 'CUST005', 'Male', 30, 'Beauty', 2,  50,  100)
 (  6, '2023-04-25', 'CUST006', 'Female', 45, 'Beauty', 1,  30,   30)
 ( 12, '2023-10-30', 'CUST012', 'Male', 35, 'Beauty', 3,  25,   75)
 ( 21, '2023-01-14', 'CUST021', 'Female', 50, 'Beauty', 1, 500,  500)
 ( 25, '2023-12-26', 'CUST025', 'Female', 64, 'Beauty', 1,  50,   50)
 ( 27, '2023-08-03', 'CUST027', 'Female', 38, 'Beauty', 2,  25,   50)
 ( 28, '2023-04-23', 'CUST028', 'Female', 43, 'Beauty', 1, 500,  500)
 ( 30, '2023-10-29', 'CUST030', 'Female', 39, 'Beauty', 3, 300,  900)
 ( 32, '2023-01-04', 'CUST032', 'Male', 30, 'Beauty', 3,  30,   90)
 ( 35, '2023-08-05', 'CUST035', 'Female', 58, 'Beauty', 3, 300,  900)
 ( 36, '2023-06-24', 'CUST036', 'Male', 52, 'Beauty', 3, 300,  900)
 ( 37, '2023-05-23', 'CUST037', 'Female', 18, 'Beauty', 3,  25,   75)
 ( 38, '2023-03-21', 'CUST038', 'Male', 38, 'Beauty', 4,  50,  200)
 ( 4

In [118]:
print("Filtro de datos sólo Clothing")
print(datos[datos["Product_Category"] == 'Clothing'])

Filtro de datos sólo Clothing
[(  2, '2023-02-27', 'CUST002', 'Female', 26, 'Clothing', 2, 500, 1000)
 (  4, '2023-05-21', 'CUST004', 'Male', 37, 'Clothing', 1, 500,  500)
 (  7, '2023-03-13', 'CUST007', 'Male', 46, 'Clothing', 2,  25,   50)
 ( 10, '2023-10-07', 'CUST010', 'Female', 52, 'Clothing', 4,  50,  200)
 ( 11, '2023-02-14', 'CUST011', 'Male', 23, 'Clothing', 2,  50,  100)
 ( 14, '2023-01-17', 'CUST014', 'Male', 64, 'Clothing', 4,  30,  120)
 ( 16, '2023-02-17', 'CUST016', 'Male', 19, 'Clothing', 3, 500, 1500)
 ( 17, '2023-04-22', 'CUST017', 'Female', 27, 'Clothing', 4,  25,  100)
 ( 19, '2023-09-16', 'CUST019', 'Female', 62, 'Clothing', 2,  25,   50)
 ( 20, '2023-11-05', 'CUST020', 'Male', 22, 'Clothing', 3, 300,  900)
 ( 22, '2023-10-15', 'CUST022', 'Male', 18, 'Clothing', 2,  50,  100)
 ( 23, '2023-04-12', 'CUST023', 'Female', 35, 'Clothing', 4,  30,  120)
 ( 24, '2023-11-29', 'CUST024', 'Female', 49, 'Clothing', 1, 300,  300)
 ( 34, '2023-12-24', 'CUST034', 'Female', 51, 'C

### Realiza operaciones de suma, resta, multiplicación y división en los datos para obtener estadísticas adicionales.

In [121]:
def obtener_total_ventas_desde_cantidad_precio_unidad(datos):
    return datos["Price_per_Unit"] * datos["Quantity"]


print("Cálculo de ventas a través de multiplicación")
total_ventas_cantidad_precio_unidad = obtener_total_ventas_desde_cantidad_precio_unidad(datos)
print(total_ventas_cantidad_precio_unidad)

Cálculo de ventas a través de multiplicación
[ 150 1000   30  500  100   30   50  100  600  200  100   75 1500  120
 2000 1500  100   50   50  900  500  100  120  300   50 1000   50  500
   30  900 1200   90  100  150  900  900   75  200  120   50   50  900
  300   25   30 1200 1500  900 1000   75   75  300  100 1500  120  900
   30 1200   50  150  200  100   50  100 2000   30 1200  300   75  300
  100 2000   90 2000  200  100  100 1500  300   60   50  200  100   90
  150   90  100  500 2000   30  500  120 2000 1000   60  600 1000  100
 1200   30  600   50   25 1000  500   50 1200   75 2000  900 1500 1500
   50  100 1500   30 1000 2000  150   50  200  120   60 2000  100   90
   50  500  600  500  600  200  900   50   50  600 1000  200 2000   30
   50 1200   50 1500   75  200  300   60   75  120   50 2000 1000  900
 2000  100 2000  600  200  100 1000   60  150 1500 1200 2000  150  300
 1500   50  900   50  120  300  100  100  100   60  300  900 1200  120
  900  200   25  200  100   75  